In [1]:
import sys
import os
sys.path.append(os.path.split(os.getcwd())[0])
from datetime import datetime
import pickle
from fast_jtnn import *
from MS_PredictModel import MS_Dataset,MS_Dataset_pickle,dataset_load
import pickle
import torch
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs

sys.path.append(os.path.split(os.getcwd())[0])

def get_timef():
    now = datetime.now()
    return now.strftime("%Y%m%d-%H%M")

print(get_timef())
device="cuda"

20190903-1742


In [2]:

from MS_PredictModel import MS_Dataset,MS_Dataset_pickle,dataset_load

VOCAB_FILE = "./MS_vocab.txt"

vocab = [x.strip("\r\n ") for x in open(VOCAB_FILE,"r")]
vocab = Vocab(vocab)

vali_dataset_path = "test_vali_data.pkl"
with open(vali_dataset_path,"rb") as f:
    vali_data = pickle.load(f)
print(vali_dataset_path,len(vali_data))


('test_vali_data.pkl', 732)


In [8]:
from ms_encoder import ms_peak_encoder,ms_peak_encoder_lstm,ms_peak_encoder_cnn
import torch.nn as nn
import torch
hidden_size = 100
latent_size = 56
depthT = 20
depthG = 3

dec_model = JTNNVAE(vocab, hidden_size, latent_size, depthT, depthG).to(device)
print dec_model
#enc_model = ms_peak_encoder_lstm(train_dataset.max_spectrum_size,output_size=latent_size,\
#        hidden_size=200,embedding_size=20,num_rnn_layers=2,bidirectional=False,dropout_rate=0.5).to('cuda')
enc_model = ms_peak_encoder_cnn(vali_data.max_spectrum_size,output_size=latent_size,\
                                 conv1_channel=64,conv2_channel=128,conv_output_channel=256,\
                                 kernel1_width=5,kernel2_width=5,conv_output_width=5,\
                                 hidden_size=200,embedding_size=10,num_rnn_layers=2,bidirectional=False,dropout_rate=0.2).to(device)
print enc_model

for param in dec_model.parameters():
    if param.dim() == 1:
        nn.init.constant_(param, 0)
    else:
        nn.init.xavier_normal_(param)

model_path = "./enc_model/model.iter-30000"
if model_path is not None:
    enc_model.load_state_dict(torch.load(model_path,map_location=device))
model_path = "./dec_model/model.iter-30000"
if model_path is not None:
    dec_model.load_state_dict(torch.load(model_path,map_location=device))

print "Model #Params: %dK" % (sum([x.nelement() for x in dec_model.parameters()]) / 1000,)
print "Model #Params: %dK" % (sum([x.nelement() for x in enc_model.parameters()]) / 1000,)

JTNNVAE(
  (jtnn): JTNNEncoder(
    (embedding): Embedding(1027, 100)
    (outputNN): Sequential(
      (0): Linear(in_features=200, out_features=100, bias=True)
      (1): ReLU()
    )
    (GRU): GraphGRU(
      (W_z): Linear(in_features=200, out_features=100, bias=True)
      (W_r): Linear(in_features=100, out_features=100, bias=False)
      (U_r): Linear(in_features=100, out_features=100, bias=True)
      (W_h): Linear(in_features=200, out_features=100, bias=True)
    )
  )
  (decoder): JTNNDecoder(
    (embedding): Embedding(1027, 100)
    (W_z): Linear(in_features=200, out_features=100, bias=True)
    (U_r): Linear(in_features=100, out_features=100, bias=False)
    (W_r): Linear(in_features=100, out_features=100, bias=True)
    (W_h): Linear(in_features=200, out_features=100, bias=True)
    (W): Linear(in_features=128, out_features=100, bias=True)
    (U): Linear(in_features=128, out_features=100, bias=True)
    (U_i): Linear(in_features=200, out_features=100, bias=True)
    (W_o)

In [9]:
from tqdm import tqdm
vali_data.batch_size = 10

sample_rate_list = [
    #[0.0,1],
    #[0.5,20],
    [1.0,1]
]

def evaluation():
    ret = []
    with torch.no_grad():
        for batch_number,batch in tqdm(enumerate(vali_data)):
            x_batch, x_jtenc_holder, x_mpn_holder, x_jtmpn_holder,x,y = batch
            x = x.to(device)
            y = y.to(device)
            result_one_data=[]
            for x_data in x_batch:
                result_one_data.append([Chem.MolToSmiles(Chem.MolFromSmiles(x_data.smiles),True)])
            for sample_rate,times in sample_rate_list:
                for No in range(times):
                    h,_ = enc_model(x,y,training=False,sample=True,sample_rate=sample_rate)
                    tree_vec = h[:,:h.shape[1]/2]
                    mol_vec  = h[:,h.shape[1]/2:]
                    for num in range(h.size()[0]):
                        predict_smiles = dec_model.decode(tree_vec[num].view(1,latent_size/2),mol_vec[num].view(1,latent_size/2),False)
                
                        #smilesの正規化
                        predict_smiles = Chem.MolToSmiles(Chem.MolFromSmiles(predict_smiles),True)
                
                        result_one_data[num].append(predict_smiles)
            ret.append(result_one_data)
    return ret
result = evaluation()
print(len(result))

0it [00:00, ?it/s]

(torch.Size([10, 310, 10]),)
(torch.Size([10, 310, 1]),)
(torch.Size([10, 310, 11]),)
(torch.Size([10, 310, 64]),)
(torch.Size([10, 310, 64]),)
(torch.Size([10, 310, 200]),)
(torch.Size([10, 200]),)


73it [19:52, 17.17s/it]

73


In [10]:
ret = []
for one in result:
    ret.extend(one)

In [11]:
len(ret)

730

In [12]:
def _re_smiles(smiles1,smiles2):
    #print(smiles1,smiles2)
    smiles1 = Chem.MolToSmiles(Chem.MolFromSmiles(smiles1),True)
    smiles2 = Chem.MolToSmiles(Chem.MolFromSmiles(smiles2),True)
    return smiles1 == smiles2

def is_structural_isomer(smiles1,smiles2):
    def Molecular_formula(smiles):
        atoms = {}
        mol = Chem.AddHs(Chem.MolFromSmiles(smiles))
        for atom in mol.GetAtoms():
            if not atom.GetSymbol() in atoms:
                atoms[atom.GetSymbol()] = 1
            else:
                atoms[atom.GetSymbol()] += 1
        return atoms
    atoms1 = Molecular_formula(smiles1)
    atoms2 = Molecular_formula(smiles2)
    return atoms1 == atoms2

def logout(one_sam,f):
    global sample_rate_list
    true_smiles = one_sam[0]
    predict_smiles = one_sam[1:]
    temp = []
    cursor = 1
    for one in one_sam:
        f.write(one+",")
    for one in predict_smiles:
        f.write(str(_re_smiles(true_smiles,one))+","+str(is_structural_isomer(true_smiles,one))+",")
    ecfp_scores = []
    maccs_scores = []
    for one in predict_smiles:
        true_mol = Chem.MolFromSmiles(true_smiles)
        predict_mol = Chem.MolFromSmiles(one)
        
        true_fingerprint = AllChem.GetMorganFingerprint(true_mol,2)
        predict_fingerprint = AllChem.GetMorganFingerprint(predict_mol,2)
        ECFP_score = DataStructs.TanimotoSimilarity(true_fingerprint,predict_fingerprint)
        ecfp_scores.append(ECFP_score)
        
        true_fingerprint = AllChem.GetMACCSKeysFingerprint(true_mol)
        predict_fingerprint = AllChem.GetMACCSKeysFingerprint(predict_mol)
        MACCS_score = DataStructs.TanimotoSimilarity(true_fingerprint,predict_fingerprint)
        maccs_scores.append(MACCS_score)
    for score in ecfp_scores:
        f.write(str(score)+",")
    f.write(",")
    for score in maccs_scores:
        f.write(str(score)+",")
    f.write(",")
        
        
    for rate,num in sample_rate_list:
        temp.append(one_sam[cursor:cursor+num])
        cursor += num
    #print(temp)
    
    
with open("area_sample.csv","w") as f:
    f.write("{},,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n".format(get_timef()))
    f.write("total sample,{}\n".format(len(ret)))
    f.write("sample_rate,number of sample\n")
    for rate,num in sample_rate_list:
        f.write("{},{}\n".format(rate,num))
    for one in ret:
        logout(one,f)
        f.write("\n")
        pass
    #logout(ret[0],f)
        